## APAN 5400 Managing Data Term Project -- Recipe Generator

### Key features of our app
1. Python Flask to get user inputs based on what ingredients they have
2. REST API to request/query from RapidAPI to get recipe information from Spoonacular
3. Store json returns in MongoDB
4. Return relevant recipes on a html webform for easy viewing

Note: This is a developmental system meant to illustrate a proof of concept for the technologies taught in class in order to [potentially] solve a real life problem.

### Instructions

1. Create a "templates" folder that is housed in the same location as this .ipynb file
2. Place base.html, search.html and recipes.html into the "templates" folder
3. Install flask and pymongo if needed
4. Ensure MongoDB is running on Docker
5. Register for a RapidAPI key on https://rapidapi.com/spoonacular/api/recipe-food-nutrition/
5. That's it and happy cooking!

Note: Please exercise caution when running the app because this is using Marcus' API key, which has a daily limit of 50 requests a day. Exceeding the daily limit would be charged to Marcus. Alternatively, we highly recommend that you register for a personal RapidAPI account with your credit card info (you won't be charged unless you exceed the limits) at https://rapidapi.com/spoonacular/api/recipe-food-nutrition/

In [1]:
# Install Flask package if needed
# !pip install flask

# Install pymongo package if needed
# !pip3 install pymongo

### Create frontend page for users to key in ingredients

1. Define API endpoint and keys
2. Create a search_page for user to key in ingredients
3. Display a random joke to spice things up

In [2]:
from flask import Flask, render_template, request

import requests
app = Flask(__name__)

url = "https://spoonacular-recipe-food-nutrition-v1.p.rapidapi.com/"

# Change API key to your own if needed
headers = {
  'x-rapidapi-host': "spoonacular-recipe-food-nutrition-v1.p.rapidapi.com",
  'x-rapidapi-key': "85e24d6ad3msh2bfead0e7b0f0f7p19ba4fjsnef67b74a1d08", 
  }

random_joke = "food/jokes/random"
find = "recipes/findByIngredients"
randomFind = "recipes/random"

@app.route('/')
def search_page():
  joke_response = str(requests.request("GET", url + random_joke, headers=headers).json()['text'])
  return render_template('search.html', joke=joke_response)

### Create a recipe search results page

1. Initialize MongoDB client
2. Create the get_recipes function to query from RapidAPI
3. Store response into MongoDB
4. Return results on a webpage that shows recipe information and photos
5. If no ingredients are entered, return 5 random recipes instead

In [3]:
from pymongo import MongoClient
from pprint import pprint
import json

client = MongoClient('localhost',27017)
db = client.apan5400

@app.route('/recipes')
def get_recipes():
    if (str(request.args['ingredients']).strip() != ""):
        # If there is a list of ingredients filled in text box -> list
        # Query recipes based on user inputs from RapidAPI
        querystring = {"number":"5","ranking":"1","ignorePantry":"false","ingredients":request.args['ingredients']}
        response = requests.request("GET", url + find, headers=headers, params=querystring).json()
        
        # Backend store in MongoDB
        collection = db.response
        collection.insert_many(response)
        
        # Query from MongoDB
        documents = list(collection.find().skip(collection.count() - 5))
        return render_template('recipes.html', recipes=documents)
    else:
        # If there are no ingredients in the input, return random recipes
        # However, we haven't got time to design the backend store for this scenario
        # But the same logic applies, where we store it in MongoDB and query data from there
        # For now, we display recipe results straight from returns from RapidAPI
        querystring = {"number":"5"}
        response = requests.request("GET", url + randomFind, headers=headers, params=querystring).json()
        return render_template('recipes.html', recipes=response['recipes'])

### Run app on localhost

In [4]:
if __name__ == '__main__':
  app.run(host='localhost', port=5002)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://localhost:5002/ (Press CTRL+C to quit)
127.0.0.1 - - [10/Dec/2021 14:11:43] "GET / HTTP/1.1" 200 -
<ipython-input-3-5c324b91b801>:21: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  documents = list(collection.find().skip(collection.count() - 5))
127.0.0.1 - - [10/Dec/2021 14:11:50] "GET /recipes?ingredients=beef%2C+tomato%2C+cheese%2C+milk HTTP/1.1" 200 -
